<a href="https://colab.research.google.com/github/shrutib55/cs_4372_Assignment_4/blob/main/Another_copy_of_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install transformers[sentencepiece]

In [ ]:
!pip install sentence-splitter


In [ ]:
import requests


In [ ]:
import textwrap

wrapper = textwrap.TextWrapper(width=80, break_long_words=False, break_on_hyphens=False)


In [ ]:
from transformers import pipeline

In [ ]:
summarization_pipeline = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import re
from sentence_splitter import SentenceSplitter

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7bb63d6a677d299c23dcaa97857152c823677ffe266feea179534b42c3838ec9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from evaluate import load

rouge = load('rouge')

In [ ]:
url = 'https://www.gutenberg.org/cache/epub/2591/pg2591.txt'
response = requests.get(url)

In [ ]:
book_text = response.text



In [ ]:
print(book_text[1078:2000])

In [ ]:
book_text_filt = book_text[1078:]

In [ ]:
print(book_text_filt[:1800])

In [ ]:
book_chapters = book_text_filt[:1800]

In [ ]:
chapter_list_x = list(book_chapters.splitlines())

chapter_list = [line.strip() for line in chapter_list_x]


In [ ]:
chapter_list.pop()

In [ ]:
chapter_list

In [ ]:
book_text_filtered = book_text_filt[1801:]

In [ ]:
def split_chapters(text, title_list):
    lines = text.splitlines()
    chapter_titles = []
    chapter_contents = []
    current_title = None
    current_content = []

    for line in lines:
        matched_title = next((title for title in title_list if title in line), None)

        if matched_title:
            if current_title is not None:
                chapter_titles.append(current_title)
                chapter_contents.append("\n".join(current_content))
                current_content = []

            current_title = matched_title.strip()

        current_content.append(line.strip())

    if current_content:
        chapter_titles.append(current_title)
        chapter_contents.append("\n".join(current_content))

    return chapter_titles, chapter_contents

chapter_titles, chapter_contents = split_chapters(book_text_filtered, chapter_list)

print(f"Chapters found: {len(chapter_contents)}")
if chapter_titles:
    print(f"First Chapter Title: {chapter_titles[1]}")
    print(f"First Chapter Text:\n{chapter_contents[1]}")


Chapters found: 69
First Chapter Title: HANS IN LUCK
First Chapter Text:
HANS IN LUCK


Some men are born to good luck: all they do or try to do comes
right--all that falls to them is so much gain--all their geese are
swans--all their cards are trumps--toss them which way you will, they
will always, like poor puss, alight upon their legs, and only move on so
much the faster. The world may very likely not always think of them as
they think of themselves, but what care they for the world? what can it
know about the matter?

One of these lucky beings was neighbour Hans. Seven long years he had
worked hard for his master. At last he said, ‘Master, my time is up; I
must go home and see my poor mother once more: so pray pay me my wages
and let me go.’ And the master said, ‘You have been a faithful and good
servant, Hans, so your pay shall be handsome.’ Then he gave him a lump
of silver as big as his head.

Hans took out his pocket-handkerchief, put the piece of silver into it,
threw it over 

In [ ]:
max_chunk_length = 512
for title, content in zip(chapter_titles, chapter_contents):
    content_chunks = [content[i:i+max_chunk_length] for i in range(0, len(content), max_chunk_length)]
    summary_text = ""
    for chunk in content_chunks:
        outputs = summarization_pipeline(chunk, max_length=50, min_length=15, clean_up_tokenization_spaces=True)
        summary_text += outputs[0]['summary_text'] + " "
        #print(outputs)

    print(f"Chapter Title: {title}")
    print(wrapper.fill(summary_text))
    print("\n" + "-"*80 + "\n")


[{'summary_text': 'A certain king had a beautiful garden, and in the garden stood a tree which bore golden apples. The king became very angry at this, and ordered the gardener to keep watch all night under the tree. The gardener set his'}]
[{'summary_text': 'The gardener at first would not let him, for fear some harm should come to him. The young man laid himself under the tree to watch. As the clock struck twelve he heard a rustling in the air, and a bird'}]
[{'summary_text': 'The golden feather was brought to the king in the morning, and all the council was called together. Everyone agreed that it was worth more than all the wealth of the kingdom. But the king said, ‘One feather is of no'}]
[{'summary_text': 'A fox tells a man how to find the golden bird. The fox tells him to go to a village and rest for the night in an inn.'}]
[{'summary_text': 'The son thought to himself, ‘What can such a beast as this know about the matter?’ So he shot his arrow at the fox; but he missed it, and i

Your max_length is set to 50, but your input_length is only 41. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


[{'summary_text': 'The coal slipped after her, hissed when she got into the water, and breathed her last. The bean, who had purposefully stayed behind on the shore, could not but laugh at the event, unable to stop.'}]
[{'summary_text': 'The bean thanked him most prettily, but as thetailor used black thread, all beans since then have a black seam. hread,and sewed her together.'}]
Chapter Title: THE STRAW, THE COAL, AND THE BEAN
A poor old woman made a fire on her hearth, and lighted it with a handful of
straw. When she was emptying the beans into the pan, one dropped without her
noticing. Soon afterwards a burning coal from the fire The coal said: ‘I
fortunately sprang out of the fire, and if I had not escaped                by
sheer force, my death would have been certain,--I should have beenburnt to
ashes’ The bean said: ‘I luckily slipped through her fingers.’ ‘But what are we
to do now?’ said the coal. ‘I think,’ answered the bean, ‘that as we have so
fortunately escaped A coal, of 

Your max_length is set to 50, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


[{'summary_text': 'Pigeons, horses, and the dogs jumped up and barked. The flies on the walls buzzed again.'}]
[{'summary_text': 'The prince and Briar Rose were married, and the wedding feastwas given. They lived happily together all their lives long.'}]
Chapter Title: BRIAR ROSE
A king and queen once upon a time reigned in a country a great way off, where
there were in those days fairies. But one day as the queen was walkingby the
side of the river, at the bottom of the The queen took pity on the littlefish,
and threw it back again into the river. It lifted its head out of the water and
said, ‘I know what your wish is,                and it shall be fulfilled, in
return The king and queen had only twelve golden dishes for them to eat. They
were forced to leave one of the fairies without asking her. After the feast was
over they gathered round in a ring and gave all their best gifts to the little
princess. One gave her goodness, another beauty, another riches, and so on till
she had al

...

Your max_length is set to 50, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


[{'summary_text': "Catherine whispered in Frederick's head that she must throw the door down soon. But he begged and prayed her not to do so, for he was sure it would betray them. ‘Here goes, however,’ said she"}]
[{'summary_text': " sound.  sound. sound. The sound of the phone ringing is the sound. of a phone ringing. It's the sound of a person talking."}]
Chapter Title: FREDERICK AND CATHERINE
There was once a man called Frederick: he had a wife whose name was Catherine.
One day Frederick said. ‘Kate! I am going to work in the fields; when I come
back I shall be. horriblyhungry so Catherine stood by with a fork and turned the
steak. Then she said to herself, ‘The steak is almost ready, I may as well go to
the cellar for the ale’ Catherine ran across the field, and away ran the dog.
But he ran faster than she, and stuck close to the steak. ‘It’s all gone, and
“what can’t be cured must When she got to the cellar stairs she saw what had
happened. ‘My stars!’ said she, ‘what shall I do t

Your max_length is set to 50, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


[{'summary_text': "'I will go home and ask if it be I, or if it is not I, they will be sure,' she thought. She ran to the door of her own house, but it was shut. She knocked at the window"}]
[{'summary_text': 'She ran out of the village, and no one has seen her since.'}]
Chapter Title: CLEVER ELSIE
There was once a man who had a daughter who was called Clever Elsie. When she
had grown up her father said: ‘We will get her married’ ‘Yes,’ said the mother,
‘if Elsie's mother said: ‘Elsie, go into the cellar and fetch some beer’ ‘Well,’
said Hans, ‘if she is not really smart, I won’t have her� Clever Elsie placed
the can before her, and turned the tap, and while the beer was running she would
not let her eyes be idle. She peered up at the wall, and after much peering here
and there, Those upstairs waited for the drink, but Clever Elsie still did not
come. Then the woman said to the servant: 'Just go down into the cellar and see
where Elsie is' The maid went and found her sitting in front E

KeyboardInterrupt: 

changed max chunk length to 700

In [ ]:
max_chunk_length = 700
for title, content in zip(chapter_titles, chapter_contents):
    content_chunks = [content[i:i+max_chunk_length] for i in range(0, len(content), max_chunk_length)]
    summary_text = ""
    for chunk in content_chunks:
        outputs = summarization_pipeline(chunk, max_length=50, min_length=15, clean_up_tokenization_spaces=True)
        summary_text += outputs[0]['summary_text'] + " "
        #print(outputs)

    print(f"Chapter Title: {title}")
    print(wrapper.fill(summary_text))
    print("\n" + "-"*80 + "\n")


In [ ]:
outputs = summarization_pipeline(content_chunks[1], max_length=50, min_length=15, clean_up_tokenization_spaces=True)
print(outputs)
print(wrapper.fill(outputs[0]['summary_text']))


[{'summary_text': 'The gardener at first would not let him, for fear some harm should come to him. The young man laid himself under the tree to watch. As the'}]
The gardener at first would not let him, for fear some harm should come to him.
The young man laid himself under the tree to watch. As the


In [ ]:
word_count = len(content_chunks[1].split())
print(word_count)

102


In [ ]:
content_chunks[1]

'as missing. Then the second son was\nordered to watch; and at midnight he too fell asleep, and in the morning\nanother apple was gone. Then the third son offered to keep watch; but\nthe gardener at first would not let him, for fear some harm should come\nto him: however, at last he consented, and the young man laid himself\nunder the tree to watch. As the clock struck twelve he heard a rustling\nnoise in the air, and a bird came flying that was of pure gold; and as\nit was snapping at one of the apples with its be'

In [ ]:
chapter_titles_filtered = chapter_titles.copy()[:2]
chapter_contents_filtered = chapter_contents.copy()[:2]

max_chunk_length = 700

candidates = []
references = []

for title, content in zip(chapter_titles_filtered, chapter_contents_filtered):
    content_chunks = [content[i:i+max_chunk_length] for i in range(0, len(content), max_chunk_length)]

    summary_text = ""

    for chunk in content_chunks:
        outputs = summarization_pipeline(
            chunk,
            max_length=50,
            min_length=15,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.5,
            num_beams=4,
            clean_up_tokenization_spaces=True
        )
        summary_text += outputs[0]['summary_text'] + " "
    candidates.append(summary_text.strip())
    references.append(content.strip())

    print(f"Chapter Title: {title}")
    print(wrapper.fill(summary_text))


    results = rouge.compute(predictions=candidates, references=references)
    print(results)
    print("\n" + "-"*80 + "\n")


Chapter Title: THE GOLDEN BIRD
A certain king had a beautiful garden, and in the garden stood a tree which bore
golden apples. These apples were always counted, and about the time when they
began to grow ripe it was found that every night one of them was gone Gardener’s
son shot an arrow at a bird that was of pure gold. But the arrow did the bird no
harm;only it dropped a golden feather from its tail. The golden feather was
brought to the king in A gardener’s eldest son set out to find the golden bird.
He came to a wood, and by the side of the wood he saw a fox sitting. The fox
said, ‘Do not shoot me, I The son thought to himself, ‘What can such a beast as
this know about the matter?’ So he shot his arrow at                the fox; but
he missed it, and it set up its tail above its back and The second son set out,
and the same thing happened to him. He met the fox, who gave him the good
advice. His eldest brother was standing at the window where merrymaking was, and
called to him to co

# Round 1

increased top_k to 100, decreased top_p to 0.85, decreased temperature to 0.6, and increased number of beams to 6

In [ ]:
chapter_titles_filtered = chapter_titles.copy()[:2]
chapter_contents_filtered = chapter_contents.copy()[:2]

max_chunk_length = 700

candidates = []
references = []

for title, content in zip(chapter_titles_filtered, chapter_contents_filtered):
    content_chunks = [content[i:i+max_chunk_length] for i in range(0, len(content), max_chunk_length)]

    summary_text = ""

    for chunk in content_chunks:
        outputs = summarization_pipeline(
            chunk,
            max_length=50,
            min_length=15,
            do_sample=True,
            top_k=100,
            top_p=0.85,
            temperature=0.6,
            repetition_penalty=1.5,
            num_beams=6,
            clean_up_tokenization_spaces=True
        )
        summary_text += outputs[0]['summary_text'] + " "
    candidates.append(summary_text.strip())
    references.append(content.strip())

    print(f"Chapter Title: {title}")
    print(wrapper.fill(summary_text))


    results = rouge.compute(predictions=candidates, references=references)
    print(results)
    print("\n" + "-"*80 + "\n")


Chapter Title: THE GOLDEN BIRD
A certain king had a beautiful garden, and in the garden stood a tree which bore
golden apples. The king became very angry at this, and ordered the gardener to
keep watch all night under the tree. The gardener set his Gardener’s son shot an
arrow at a bird that was of pure gold. But the arrow did the bird no harm;only
it dropped a golden feather from its tail. The golden feather was brought to the
king in A gardener’s eldest son set out to find the golden bird. He came to a
wood, and by the side of the wood he saw a fox sitting. The fox said, ‘Do not
shoot me, I The son thought to himself, ‘What can such a beast as this know
about the matter?’ So he shot his arrow at                the fox; but he missed
it, and it set up its tail above its back and The second son set out, and the
same thing happened to him. He met the fox, who gave him the good advice. But
when he came to the two inns, his eldest brother was standing at the window. He
could The son follo

## Round 2

Increase the max_length to 60, increased the top_k to 150, decreased the top_p to 0.8, and increased the num_beams to 8

In [ ]:
chapter_titles_filtered = chapter_titles.copy()[:2]
chapter_contents_filtered = chapter_contents.copy()[:2]

max_chunk_length = 700

candidates = []
references = []

for title, content in zip(chapter_titles_filtered, chapter_contents_filtered):
    content_chunks = [content[i:i+max_chunk_length] for i in range(0, len(content), max_chunk_length)]

    summary_text = ""

    for chunk in content_chunks:
        outputs = summarization_pipeline(
            chunk,
            max_length=60,
            min_length=15,
            do_sample=True,
            top_k=150,
            top_p=0.8,
            temperature=0.6,
            repetition_penalty=1.5,
            num_beams=8,
            clean_up_tokenization_spaces=True
        )
        summary_text += outputs[0]['summary_text'] + " "
    candidates.append(summary_text.strip())
    references.append(content.strip())

    print(f"Chapter Title: {title}")
    print(wrapper.fill(summary_text))


    results = rouge.compute(predictions=candidates, references=references)
    print(results)
    print("\n" + "-"*80 + "\n")


Chapter Title: THE GOLDEN BIRD
A certain king had a beautiful garden, and in the garden stood a tree which bore
golden apples. About the time when they began to grow ripe it was found that
every night one                of them was gone. The king became very angry at
this, and ordered the gardener to keep The gardener’s son shot an arrow at a
bird that was of pure gold. The arrow did the bird no harm;only it dropped a
golden feather from its tail. Everyone agreed that it was worth more than all
the wealth of the kingdom. But the king said, A gardener’s eldest son set out to
find the golden bird. When he had gone but a little way, he came to a wood, and
by the side of the wood he saw a fox sitting. The fox said, ‘Do not shoot me, I
will The son thought to himself, ‘What can such a beast as this know about the
matter?’ So he shot his arrow at                the fox; but he missed it, and
it set up its tail above its back. Then he went his way, and in the evening The
second son set out, a

## Round 3

In [ ]:
chapter_titles_filtered = chapter_titles.copy()[:2]
chapter_contents_filtered = chapter_contents.copy()[:2]

max_chunk_length = 700

candidates = []
references = []

for title, content in zip(chapter_titles_filtered, chapter_contents_filtered):
    content_chunks = [content[i:i+max_chunk_length] for i in range(0, len(content), max_chunk_length)]

    summary_text = ""

    for chunk in content_chunks:
        outputs = summarization_pipeline(
            chunk,
            max_length=60,
            min_length=15,
            do_sample=True,
            top_k=200,
            top_p=0.7,
            temperature=0.6,
            repetition_penalty=1.5,
            num_beams=10,
            clean_up_tokenization_spaces=True
        )
        summary_text += outputs[0]['summary_text'] + " "
    candidates.append(summary_text.strip())
    references.append(content.strip())

    print(f"Chapter Title: {title}")
    print(wrapper.fill(summary_text))


    results = rouge.compute(predictions=candidates, references=references)
    print(results)
    print("\n" + "-"*80 + "\n")


Chapter Title: THE GOLDEN BIRD
A certain king had a beautiful garden, and in the garden stood a tree which bore
golden apples. About the time when they began to grow ripe it was found that
every night one                of them was gone. The king became very angry at
this, and ordered the gardener to keep The gardener’s son shot an arrow at a
bird that was of pure gold. The arrow did the bird no harm;only it dropped a
golden feather from its tail. Everyone agreed that it was worth more than all
the wealth of the kingdom. But the king said, A gardener’s eldest son set out to
find the golden bird. When he had gone but a little way, he came to a wood, and
by the side of the wood he saw a fox sitting. The fox said, ‘Do not shoot me, I
will The son thought to himself, ‘What can such a beast as this know about the
matter?’ So he shot his arrow at                the fox; but he missed it, and
it set up its tail above its back. Then he went his way, and in the evening The
second son set out, a